In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

название на русском, год, название в оригинале, страна, жанр, актеры (не убираются актеры озвучки, пускай останутся через / ), слоган, режиссер, возр ограничения, длительность, описание, рейтинг на кинопоиске, рейтинг на imdb, ссылка на трейлер

In [2]:
import pandas as pd

In [3]:
'''
writer = pd.ExcelWriter(name_rus + '.xlsx')
movie.to_excel(writer,'Sheet1')
writer.save()
'''

"\nwriter = pd.ExcelWriter(name_rus + '.xlsx')\nmovie.to_excel(writer,'Sheet1')\nwriter.save()\n"

In [4]:
def get_name_rus(soup):
    name_rus = ''
    if soup.find('h1', {'class': 'moviename-big'}):
        name_rus = soup.find('h1', {'class': 'moviename-big'}).text
    return name_rus

In [5]:
def get_year(soup):
    year = ''
    if soup.find_all('div', {'style': 'position: relative'}):
        year = soup.find_all('div', {'style': 'position: relative'})[0].text[1:-1].replace('\n', ' ')
    return year

In [6]:
def get_name_eng(soup):
    name_eng = ''
    if soup.find('span', {'itemprop': 'alternativeHeadline'}):
        name_eng = soup.find('span', {'itemprop': 'alternativeHeadline'}).text
    return name_eng

In [7]:
def get_country(soup):
    country = ''
    if soup.find_all('div', {'style': 'position: relative'}):
        country = soup.find_all('div', {'style': 'position: relative'})[1].text[1:-1] 
    return country

In [8]:
def get_genre(soup):
    genre = ''
    if soup.find('span', {'itemprop': 'genre'}):
        genre = soup.find('span', {'itemprop': 'genre'}).text
    return genre

In [9]:
def get_actors(soup):
    actors = ''
    for i in range (len(soup.find_all('li', {'itemprop': 'actors'}))):
        actors += soup.find_all('li', {'itemprop': 'actors'})[i].text + ', '
    #actors = actors[:-7].replace(', ...,', ' /')
    return actors[:-7].replace(', ...,', ' /')

In [10]:
def get_slogan(soup):
    #slogan = ''
    if soup.find('td', {'style': 'color: #555'}):
        return soup.find('td', {'style': 'color: #555'}).text
    return ''

In [11]:
def get_director(soup):
    if soup.find('td', {'itemprop': 'director'}):
        return soup.find('td', {'itemprop': 'director'}).text
    return ''

In [12]:
def get_age_limit(soup):
    if soup.find('span', {'style': 'margin-left: 3px'}):
        return soup.find('span', {'style': 'margin-left: 3px'}).text
    return ''

In [13]:
def get_duration(soup):
    if soup.find('td', {'class': 'time'}):
        return soup.find('td', {'class': 'time'}).text
    return ''

In [14]:
def get_description(soup):
    if soup.find('div', {'class': 'brand_words film-synopsys'}):
        return soup.find('div', {'class': 'brand_words film-synopsys'}).text.replace('\xa0', ' ').replace('\x97', '').replace('\x85', '')
    return ''

In [15]:
def get_rating_kinopoisk(soup):
    if soup.find('span', {'class': 'rating_ball'}):
        return soup.find('span', {'class': 'rating_ball'}).text
    return ''

In [16]:
def get_rating_imdb(soup):
    if soup.find('div', {'style': 'color:#999;font:100 11px tahoma, verdana'}):
        rating_imdb_str = soup.find('div', {'style': 'color:#999;font:100 11px tahoma, verdana'}).text
        return rating_imdb_str[rating_imdb_str.find(':')+2 : rating_imdb_str.find('(')-1]
    return ''

In [17]:
def get_link_on_trailer(soup):
    if soup.find('a', {'id': 'movie-trailer-block-link'}):
        short_link = soup.find('a', {'id': 'movie-trailer-block-link'}).get('href')
        if short_link.rfind('-') > 0:
            short_link = short_link[:short_link[1:].find('/')+2] + short_link[short_link.rfind('-')+1:]
        return 'https://www.kinopoisk.ru' + short_link
    return ''

In [18]:
from user_agent import generate_user_agent
from time import sleep
import numpy as np

proxies_list = [{'http':'http://'+i} for i in ['67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421',
                '111.68.99.42:8080','80.241.219.66:3128',
                '201.20.94.106:8080','216.229.120.173:45554',
                '116.58.247.31:3128','103.9.115.142:3128',
                '82.164.99.193:10200','80.188.79.138:8080',
                '36.75.113.224:8080',
                '1.20.204.163:8080','97.77.49.151:45554',
                '178.54.44.24:8080',
                '65.182.136.153:45554', '111.76.129.223:808',
                 '203.142.81.205'+':'+'8080', 
                '42.202.35.185'+':'+'8118', '189.16.249.114'+':'+'8080',
                '66.162.122.24'+':'+'8080']]


def get_request(movie_id):
    sleep(np.random.random())
    url = 'https://www.kinopoisk.ru/film/' + str(movie_id)
    return requests.get(url, headers={'User-Agent': generate_user_agent()}, 
                        proxies=proxies_list[np.random.randint(len(proxies_list))])

In [27]:
def movie_scraping(movie_id):
    r = get_request(movie_id)
    soup = BeautifulSoup(r.text, 'html.parser')
    name_rus = get_name_rus(soup)
    year = get_year(soup)
    name_eng = get_name_eng(soup)
    country = get_country(soup)
    genre = get_genre(soup)
    actors = get_actors(soup)
    slogan = get_slogan(soup)
    director = get_director(soup)
    age_limit = get_age_limit(soup)
    duration = get_duration(soup)
    description = get_description(soup)
    rating_kinopoisk = get_rating_kinopoisk(soup)
    rating_imdb = get_rating_imdb(soup)
    link_on_trailer = get_link_on_trailer(soup)
    
    movie = pd.Series()
    movie['id'] = movie_id
    movie['Ссылка'] = 'https://www.kinopoisk.ru/film/' + str(movie_id)
    movie['Название'] = name_rus 
    movie['Год выпуска'] = year
    movie['Оригинальное название'] = name_eng
    movie['Жанр'] = genre
    movie['Актеры'] = actors
    movie['Страна'] = country
    movie['Слоган'] = slogan
    movie['Режиссер'] = director
    movie['Возрастные ограничения'] = age_limit
    movie['Длительность'] = duration
    movie['Описание'] = description
    movie['Рейтинг на Кинопоиске'] = rating_kinopoisk
    movie['Рейтинг на IMDB'] = rating_imdb
    movie['Ссылка на трейлер'] = link_on_trailer
    
    return movie

In [30]:
movie_scraping(770)

id                                                                      770
Ссылка                                    https://www.kinopoisk.ru/film/770
Название                                          Одиннадцать друзей Оушена
Год выпуска                                                            2001
Оригинальное название                                        Ocean's Eleven
Жанр                                                      триллер, криминал
Актеры                    Джордж Клуни, Брэд Питт, Мэтт Дэймон, Джулия Р...
Страна                                                                  США
Слоган                                                «Делайте ваши ставки»
Режиссер                                                   Стивен Содерберг
Возрастные ограничения                           зрителям, достигшим 12 лет
Длительность                                               116 мин. / 01:56
Описание                  После выхода из тюрьмы вора Дэнни Оушена не пр...
Рейтинг на К

In [25]:
movie_scraping(891337)

id                                                                   891337
Ссылка                                 https://www.kinopoisk.ru/film/891337
Название                                             11.22.63 (мини-сериал)
Год выпуска                                                  2016 (1 сезон)
Оригинальное название                                              11.22.63
Жанр                                   фантастика, триллер, драма, детектив
Актеры                    Джеймс Франко, Сара Гадон, Джордж МакКэй, Крис...
Страна                                                                  США
Слоган                      «When you fight the past, the past fights back»
Режиссер                     Джеймс Стронг, Фред Туа, Джон Дэвид Коулз, ...
Возрастные ограничения                           зрителям, достигшим 16 лет
Длительность                                                60 мин. / 01:00
Описание                  Учитель английского языка отправляется в прошл...
Рейтинг на К

In [26]:
movie_scraping(464963)

id                                                                   464963
Ссылка                                 https://www.kinopoisk.ru/film/464963
Название                                 Игра престолов (сериал 2011 – ...)
Год выпуска                                                2011 (8 сезонов)
Оригинальное название                                       Game of Thrones
Жанр                         фэнтези, боевик, драма, мелодрама, приключения
Актеры                    Питер Динклэйдж, Лина Хиди, Эмилия Кларк, Кит ...
Страна                                                  США, Великобритания
Слоган                                                  «Победа или смерть»
Режиссер                       Дэвид Наттер, Алан Тейлор, Алекс Грейвз, ...
Возрастные ограничения                           зрителям, достигшим 18 лет
Длительность                                                        55 мин.
Описание                  К концу подходит время благоденствия, и лето, ...
Рейтинг на К